## Embedding на мед. данных 

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
# легковесная встраиваемая векторная бд
from langchain_community.vectorstores import Chroma

# модель для эмбеддингов, при первом запуске скачается в ~/.cache/huggingface/hub
# https://huggingface.co/sergeyzh/LaBSE-ru-turbo
embeddings = HuggingFaceEmbeddings(
            model_name= "sergeyzh/LaBSE-ru-turbo",
            model_kwargs={'device': 'cpu'}
        )

# embeddings.embed_documents(["Зеленый слоник", "покушать принес"])

In [1]:
cardiology_tests = [
    {
        "query": "Какие целевые значения артериального давления при гипертонии?",
        "relevant_chunk": """
        Гипертоническая болезнь - хроническое заболевание, характеризующееся стойким повышением 
        артериального давления выше 140/90 мм рт.ст. Целевое АД для большинства пациентов: 
        < 140/90 мм рт.ст. Для пациентов с сахарным диабетом и хронической болезнью почек 
        целевой уровень < 130/80 мм рт.ст.
        """,
        "expected_answer_contains": ["< 140/90 мм рт.ст", "целевое АД", "для большинства пациентов"]
    },
    {
        "query": "Препараты первой линии для лечения гипертонической болезни",
        "relevant_chunk": """
        Основные классы антигипертензивных препаратов: ингибиторы АПФ (эналаприл, лизиноприл, рамиприл), 
        блокаторы кальциевых каналов (амлодипин, нифедипин), бета-блокаторы (бисопролол, метопролол), 
        диуретики (гидрохлоротиазид, индапамид). Препараты первой линии: ингибиторы АПФ и блокаторы 
        кальциевых каналов.
        """,
        "expected_answer_contains": ["ингибиторы АПФ", "блокаторы кальциевых каналов", "эналаприл", "амлодипин"]
    }
]

endocrinology_tests = [
    {
        "query": "Диагностические критерии сахарного диабета 2 типа",
        "relevant_chunk": """
        Сахарный диабет 2 типа диагностируется при: уровне глюкозы плазмы натощак ≥ 7.0 ммоль/л, 
        глюкозе плазмы через 2 часа после ПГТТ ≥ 11.1 ммоль/л, гликированном гемоглобине (HbA1c) ≥ 6.5%. 
        Для подтверждения требуется дважды проведенный анализ в разные дни.
        """,
        "expected_answer_contains": ["глюкоза натощак ≥ 7.0 ммоль/л", "HbA1c ≥ 6.5%", "ПГТТ ≥ 11.1 ммоль/л"]
    },
    {
        "query": "Какая начальная доза метформина при диабете?",
        "relevant_chunk": """
        Метформин - препарат первой линии при сахарном диабете 2 типа. Начальная доза 500-850 мг 1-2 раза в сутки. 
        Максимальная суточная доза 2000-2550 мг. Принимается во время или после еды для уменьшения 
        gastrointestinal побочных эффектов.
        """,
        "expected_answer_contains": ["500-850 мг", "начальная доза", "во время или после еды"]
    }
]

emergency_medicine_tests = [
    {
        "query": "Неотложная терапия при остром коронарном синдроме",
        "relevant_chunk": """
        При остром коронарном синдроме неотложная терапия включает: аспирин 300 мг разжевать, 
        клопидогрел 300-600 мг нагрузочная доза, нефракционированный гепарин или низкомолекулярный гепарин. 
        При боли - морфин 2-4 мг в/в. Кислородотерапия при сатурации < 90%.
        """,
        "expected_answer_contains": ["аспирин 300 мг", "клопидогрел", "гепарин", "морфин"]
    },
    {
        "query": "В чем разница между ИМпST и ИМбпST?",
        "relevant_chunk": """
        Инфаркт миокарда с подъемом сегмента ST (ИМпST) характеризуется подъемом сегмента ST на ЭКГ 
        и требует экстренной реперфузии (ЧКВ или тромболизис). Инфаркт миокарда без подъема сегмента ST 
        (ИМбпST) проявляется депрессией сегмента ST или инверсией зубца Т, тактика - медикаментозная терапия 
        и плановая коронарография.
        """,
        "expected_answer_contains": ["подъем сегмента ST", "экстренная реперфузия", "депрессия сегмента ST", "медикаментозная терапия"]
    }
]

complex_tests = [
    {
        "query": "Ведение пациента с гипертонией и сердечной недостаточностью",
        "relevant_chunk": [
            """
            При гипертонической болезни целевое АД < 140/90 мм рт.ст. Препараты выбора: 
            ингибиторы АПФ, блокаторы кальциевых каналов, диуретики.
            """,
            """
            При хронической сердечной недостаточности основная терапия: ИАПФ/БРА, бета-блокаторы, 
            антагонисты альдостерона. При сочетании с гипертонией предпочтение отдается ИАПФ и бета-блокаторам.
            """
        ],
        "expected_answer_contains": ["ИАПФ", "бета-блокаторы", "целевое АД < 140/90", "хроническая сердечная недостаточность"]
    },
    {
        "query": "Дифференциальная диагностика болей в груди",
        "relevant_chunk": [
            """
            Острый коронарный синдром: давящая боль за грудиной, иррадиирует в левую руку, шею, челюсть, 
            сопровождается одышкой, потливостью. Длительность от 20 минут.
            """,
            """
            Перикардит: острая боль, усиливается при кашле, глубоком дыхании, в положении лежа, 
            уменьшается при наклоне вперед. Шум трения перикарда при аускультации.
            """,
            """
            ТЭЛА: острая боль, сочетается с одышкой, тахикардией, может быть кровохарканье. 
            Факторы риска: иммобилизация, онкология, тромбофилия.
            """
        ],
        "expected_answer_contains": ["острый коронарный синдром", "перикардит", "ТЭЛА", "дифференциальная диагностика"]
    }
]

In [2]:

combed = cardiology_tests + endocrinology_tests + emergency_medicine_tests + complex_tests
texts = []
metadata = []
query = []
for el in combed:
    query.append(el.get("query"))
    texts.append(el.get("relevant_chunk") if isinstance(el.get("relevant_chunk"), str) else " ".join(el.get("relevant_chunk")))
    metadata.append({"source": "doc-1", "page": 1})

vectorstore = Chroma(
    collection_name="medical_docs",
    embedding_function=embeddings,
    persist_directory="./chroma_db"
)


NameError: name 'Chroma' is not defined

In [9]:
vectorstore.delete_collection()

In [12]:
vectorstore.add_texts(texts, metadata)

['cf7eeb19-bec5-44b0-a6f9-48c2145180a4',
 'd685acf3-aaa8-4627-a978-f0bb73fbd039',
 'a1c9c967-8d7d-4c67-bfa2-30844dd4e737',
 '1be3f064-eb64-4cf0-bf5d-9500abb67daf',
 '56374ca2-d2db-456e-ace1-e9814439abf7',
 'f669d1f8-274c-4dc0-aedf-89cc4037e7c5',
 '7ee19632-79f0-433f-86eb-d9bdc28e727c',
 '46f3f3c8-a729-4b70-ad89-c13335c8be9d']

In [7]:
from IPython.display import display

In [6]:
i = 1
print(q := query[i])
display(ans := vectorstore.similarity_search_with_relevance_scores(query=query[i], k=5))

Препараты первой линии для лечения гипертонической болезни


[(Document(metadata={'source': 'doc-1', 'page': 1}, page_content='\n            При гипертонической болезни целевое АД < 140/90 мм рт.ст. Препараты выбора: \n            ингибиторы АПФ, блокаторы кальциевых каналов, диуретики.\n             \n            При хронической сердечной недостаточности основная терапия: ИАПФ/БРА, бета-блокаторы, \n            антагонисты альдостерона. При сочетании с гипертонией предпочтение отдается ИАПФ и бета-блокаторам.\n            '),
  0.6844239544881345),
 (Document(metadata={'page': 1, 'source': 'doc-1'}, page_content='\n        Основные классы антигипертензивных препаратов: ингибиторы АПФ (эналаприл, лизиноприл, рамиприл), \n        блокаторы кальциевых каналов (амлодипин, нифедипин), бета-блокаторы (бисопролол, метопролол), \n        диуретики (гидрохлоротиазид, индапамид). Препараты первой линии: ингибиторы АПФ и блокаторы \n        кальциевых каналов.\n        '),
  0.6399000237212261),
 (Document(metadata={'page': 1, 'source': 'doc-1'}, page_con

In [7]:
ans = [el[0].page_content for el in ans]
ans

['\n            При гипертонической болезни целевое АД < 140/90 мм рт.ст. Препараты выбора: \n            ингибиторы АПФ, блокаторы кальциевых каналов, диуретики.\n             \n            При хронической сердечной недостаточности основная терапия: ИАПФ/БРА, бета-блокаторы, \n            антагонисты альдостерона. При сочетании с гипертонией предпочтение отдается ИАПФ и бета-блокаторам.\n            ',
 '\n        Основные классы антигипертензивных препаратов: ингибиторы АПФ (эналаприл, лизиноприл, рамиприл), \n        блокаторы кальциевых каналов (амлодипин, нифедипин), бета-блокаторы (бисопролол, метопролол), \n        диуретики (гидрохлоротиазид, индапамид). Препараты первой линии: ингибиторы АПФ и блокаторы \n        кальциевых каналов.\n        ',
 '\n        Метформин - препарат первой линии при сахарном диабете 2 типа. Начальная доза 500-850 мг 1-2 раза в сутки. \n        Максимальная суточная доза 2000-2550 мг. Принимается во время или после еды для уменьшения \n        gastro

### cross-encoder для re-rank

In [8]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# https://huggingface.co/ncbi/MedCPT-Cross-Encoder
tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Cross-Encoder")
model = AutoModelForSequenceClassification.from_pretrained("ncbi/MedCPT-Cross-Encoder")



# combine query article into pairs
pairs = [[q, article] for article in ans]

with torch.no_grad():
    encoded = tokenizer(
        pairs,
        truncation=True,
        padding=True,
        return_tensors="pt",
        max_length=512,
    )

    logits = model(**encoded).logits.squeeze(dim=1)
    probabilities = F.softmax(logits, dim=0)
    
    print(logits)
    print(probabilities)

tensor([15.7108, 15.8161, 15.7280, 15.7460, 15.7298])
tensor([0.1929, 0.2144, 0.1963, 0.1998, 0.1966])


In [11]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
scores = model.predict(pairs)
scores

array([ 4.4234962,  1.6039315, -1.2160715, -5.8425174, -2.1149385],
      dtype=float32)

In [15]:
vectorstore.delete_collection()

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# 1. Загрузка модели и токенизатора
model_name = "jinaai/jina-embeddings-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# 2. Переместить на GPU, если доступен
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)




c:\Users\Очир\Desktop\Учёба\RAG_LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


XLMRobertaLoRA(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): ParametrizedEmbedding(
        250002, 1024, padding_idx=1
        (parametrizations): ModuleDict(
          (weight): ParametrizationList(
            (0): LoRAParametrization()
          )
        )
      )
      (token_type_embeddings): ParametrizedEmbedding(
        1, 1024
        (parametrizations): ModuleDict(
          (weight): ParametrizationList(
            (0): LoRAParametrization()
          )
        )
      )
    )
    (emb_drop): Dropout(p=0.1, inplace=False)
    (emb_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): XLMRobertaEncoder(
      (layers): ModuleList(
        (0-23): 24 x Block(
          (mixer): MHA(
            (rotary_emb): RotaryEmbedding()
            (Wqkv): ParametrizedLinearResidual(
              in_features=1024, out_features=3072, bias=True
              (parametrizations): ModuleDict(
                (weight): 

In [4]:
# 3. Функция для генерации эмбеддингов
def encode(texts, task="retrieval.passage", max_length=8192, dimensions=None):
    """
    Генерирует эмбеддинги с использованием jina-embeddings-v3.
    
    Args:
        texts: список строк
        task: одна из ['retrieval.query', 'retrieval.passage', 'separation', 'classification', 'text-matching']
        max_length: макс. длина последовательности (до 8192)
        dimensions: опционально — размерность Matryoshka (например, 512, 256). Полная — 1024.
    """
    # Добавляем префикс с задачей
    prefixed = [f"<{task}>{text}" for text in texts]
    
    # Токенизация
    batch = tokenizer(
        prefixed,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        add_special_tokens=True
    ).to(device)
    
    # Прямой проход
    with torch.no_grad():
        outputs = model(**batch)
        last_hidden = outputs.last_hidden_state  # [batch, seq_len, 1024]
    
    # Mean pooling
    attention_mask = batch['attention_mask']
    mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    sum_embeddings = torch.sum(last_hidden * mask, dim=1)
    sum_mask = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled = sum_embeddings / sum_mask  # [batch, 1024]
    
    # Matryoshka truncation
    if dimensions is not None:
        mean_pooled = mean_pooled[:, :dimensions]
    
    # Нормализация (обязательна для cosine similarity в pgvector!)
    embeddings = F.normalize(mean_pooled, p=2, dim=1)
    
    return embeddings.cpu().numpy()

queries = ["Какие симптомы при пневмонии?"]
documents = [
    "Пневмония — воспаление лёгок, сопровождающееся кашлем, лихорадкой и одышкой.",
    "Гастрит — воспаление желудка.",
    "Пневмония сопровождается следующими симптомами: кашель, одышка и лихорадка",
    "Симптомы туберкулеза это древовидное образование на рентгене легких",
    "Пневматичсекий перфоратор продаётся на рынке со скидкой"
]

# Эмбеддинги запроса ( task="retrieval.query")
query_emb = encode(queries, task="retrieval.query")

# Эмбеддинги документов (task="retrieval.passage")
doc_embs = encode(documents, task="retrieval.passage")


In [7]:
(query_emb)

array([[ 0.05565951,  0.00634775, -0.0643283 , ...,  0.02643578,
        -0.02114508, -0.02042487]], shape=(1, 1024), dtype=float32)

In [10]:
import numpy as np

a = np.array([[1, 2, 3]])
b = np.array([[4, 5, 6], [1, 2, 1]])
a @ b.T

array([[32,  8]])